## Truncation function w.r.t symmetry sectors

In [ ]:
struct TruncationDimensionDegenerate
    dim::Int
    rtol::Real
end
truncdimdeg(d::Int, t::Real) = TruncationDimensionDegenerate(d,t)

function _truncate!(V::TensorKit.SectorVector, trunc::TruncationDimensionDegenerate, p=2)
    I = keytype(V)
    S = real(eltype(valtype(V)))
    truncdim = TensorKit.SectorDict{I,Int}(c => length(v) for (c, v) in V)

    last_cut_sv = 0.0 #define variable for the last SV we cut.

    while sum(dim(c) * d for (c, d) in truncdim) > trunc.dim
        cmin = TensorKit._findnexttruncvalue(V, truncdim, p)
        cmin === nothing && break

        last_cut_sv = V[cmin][end] #remember last SV that you cut

        truncdim[cmin] -= 1 
    end

    if last_cut_sv == 0.0
        
    else

        cmin = TensorKit._findnexttruncvalue(V, truncdim, p)
        isnothing(cmin) && return truncdim #if there is nothing left to truncate, return ...
        
        largest_sv_remaining = V[cmin][truncdim[cmin]] #find the smallest SV remaining

        while isapprox(largest_sv_remaining, last_cut_sv; rtol = trunc.rtol) #here one compares - the rtol could also be passed in a new TruncationDimensionDegenerate. trunc.rtol
            #display("I am cutting extra!")
            truncdim[cmin] -= 1 #if the smallest remaining SV is roughly degenerate with the last one we cut, cut it as well. 
            cmin = TensorKit._findnexttruncvalue(V, truncdim, p) #find the next one and see if it is degenerate as well.
            isnothing(cmin) && break 
            largest_sv_remaining = V[cmin][truncdim[cmin]] #new smallest SV that is remaining
        end
    end

    truncerr = TensorKit._norm((c => view(v, (truncdim[c] + 1):length(v)) for (c, v) in V), p,
                     zero(S))
    for (c, v) in V
        TensorKit.resize!(v, truncdim[c])
    end
    return V, truncerr
end